## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-13-13-00 +0000,wsj,Elliott Takes Stake in London Stock Exchange O...,https://www.wsj.com/finance/elliott-takes-stak...
1,2026-02-11-13-11-17 +0000,nyt,NATO Is Expected to Step Up Arctic Security. H...,https://www.nytimes.com/2026/02/11/world/europ...
2,2026-02-11-13-02-38 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/11/us/faa...
3,2026-02-11-13-02-06 +0000,nyt,9 Killed and 25 Injured in Shootings in Tumble...,https://www.nytimes.com/2026/02/11/world/canad...
4,2026-02-11-13-01-00 +0000,wsj,Kraft Heinz Pauses Plan to Split Into Two Comp...,https://www.wsj.com/business/retail/kraft-hein...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2490/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
52,trump,56
207,epstein,24
20,canada,17
45,guthrie,16
435,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
155,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,171
33,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...,127
153,2026-02-11-03-51-40 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,118
276,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,110
279,2026-02-10-20-48-27 +0000,bbc,Ex-police chief said Trump told him in 2006 'e...,https://www.bbc.com/news/articles/cx2407jrn83o...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
155,171,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
168,77,2026-02-11-02-45-29 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
33,71,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...
115,64,2026-02-11-09-03-03 +0000,nypost,Tumbler Ridge students describe terror as ‘gun...,https://nypost.com/2026/02/11/world-news/tumbl...
329,43,2026-02-10-18-38-47 +0000,nypost,Indicted Dem asks ICE acting director Todd Lyo...,https://nypost.com/2026/02/10/us-news/indicted...
123,40,2026-02-11-08-37-08 +0000,wsj,Jobs Report Today: Dow Futures Inch Up; Dollar...,https://www.wsj.com/livecoverage/jobs-report-u...
55,37,2026-02-11-11-09-51 +0000,nypost,Ex-Prince Andrew’s gushing emails to Epstein b...,https://nypost.com/2026/02/11/world-news/ex-pr...
330,37,2026-02-10-18-37-00 +0000,wsj,"Republican Sen. Susan Collins, the Maine centr...",https://www.wsj.com/politics/policy/susan-coll...
204,35,2026-02-11-00-12-21 +0000,latimes,LAPD to train their body cameras on immigratio...,https://www.latimes.com/california/story/2026-...
272,34,2026-02-10-21-15-41 +0000,nypost,LI parents who tragically lost hockey star son...,https://nypost.com/2026/02/10/us-news/li-paren...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
